# Imports

In [ ]:
import cv2
import numpy as np
from google.colab import drive
# drive.mount('/content/drive')

# Reading file

In [3]:
car = np.load('/content/car2.npy', mmap_mode='r')

# Initial template

In [4]:
templates = [ [ [50,110], [160,160] ] ]

# Get Templates

In [5]:
def jacobian_cal(updated_rect, sobel_x, sobel_y):
  A = []
  for j in range(updated_rect[1],updated_rect[3]):
    for k in range(updated_rect[0],updated_rect[2]):
      delta_I = np.array([sobel_x[j][k], sobel_y[j][k]])
      jacobian_mat = np.array([[k,0,j,0,1,0],[0,k,0,j,0,1]])
      matrix = np.matmul(delta_I, jacobian_mat)
      A.append(matrix)
  return A

In [6]:
def Lucas_Kanade(I, p_prev, diff, T, curr_rect, threshold, max_iteration):
  sobel_x = cv2.Sobel(I, cv2.CV_64F, dx = 1, dy = 0, ksize = 5)
  sobel_y = cv2.Sobel(I, cv2.CV_64F, dx = 0, dy = 1, ksize = 5)

  count = 0
  while (diff >= threshold) and count <= max_iteration:
    params = np.array([[1+p_prev[0], p_prev[2], p_prev[4]], [p_prev[1], 1+p_prev[3], p_prev[5]]])

    tl = np.array([[curr_rect[0]],[curr_rect[1]],[1]])
    br = np.array([[curr_rect[2]],[curr_rect[3]],[1]])

    tl_dash = np.dot(params,tl)
    tl_x, tl_y = max(0, tl_dash[0][0]), max(0, tl_dash[1][0])

    br_x, br_y = min(I.shape[1], tl_x+110), min(I.shape[0], tl_y+50)

    updated_rect = [int(tl_x), int(tl_y), int(br_x), int(br_y)]
    warped_I = cv2.warpAffine(src=I,M=np.array(params, dtype=np.float32),dsize=(I.shape[1],I.shape[0]))
    warped_I = warped_I[updated_rect[1]:updated_rect[3], updated_rect[0]:updated_rect[2]]
    curr_frame_cropped_h, curr_frame_cropped_w = warped_I.shape

    warped_I = warped_I.reshape(((warped_I.shape[0]*warped_I.shape[1]),1))

    T_dash = T.copy()
    T_dash = T_dash[0:curr_frame_cropped_h, 0:curr_frame_cropped_w]

    T_dash = T_dash.reshape(((T_dash.shape[0]*T_dash.shape[1]),1))
    error = T_dash.astype(int) - warped_I.astype(int)

    A = jacobian_cal(updated_rect, sobel_x, sobel_y)
    
    Hessian = np.matmul(np.transpose(np.array(A)),np.array(A))
    Hessian_inv = np.linalg.pinv(Hessian)
    delta_P = np.matmul(np.matmul(Hessian_inv,np.transpose(np.array(A))),error)

    p_prev += delta_P.reshape((-1))
    diff = np.linalg.norm(delta_P)

    count += 1

  return params

In [7]:
def tracker(T, curr_rect, frames, threshold, max_iteration):

  for i in range(1,frames.shape[2]):
      
    p = np.zeros(6)
    p_prev = p
    
    diff = np.inf
    
    I = frames[:,:,i]
    
    params = Lucas_Kanade(I, p_prev, diff, T, curr_rect, threshold, max_iteration)
    
    tl = np.array([[curr_rect[0]],[curr_rect[1]],[1]])
    br = np.array([[curr_rect[2]],[curr_rect[3]],[1]])

    tl_dash = np.dot(params,tl)
    tl_dash = tl_dash.reshape((1,2))

    br_dash = np.dot(params, br)
    br_dash = br_dash.reshape((1,2))

    tl_x, tl_y = max(0, tl_dash[0][0]), max(0, tl_dash[0][1])
    br_x, br_y = min(I.shape[1], tl_x+108), min(I.shape[0], tl_y+53)

    curr_rect = [int(tl_x), int(tl_y), int(br_x), int(br_y)]

    templates.append([[int(tl_x),int(tl_y)],[int(br_x),int(br_y)]])

    print(i)

  return templates

In [8]:
def get_templates(frames, templates, threshold, max_iteration):
    
    I_dash = frames[:,:,0]
    T = I_dash[templates[0][0][1]:templates[0][1][1], templates[0][0][0]:templates[0][1][0]] 
    
    curr_rect = [templates[0][0][0], templates[0][0][1], templates[0][1][0], templates[0][1][1]]

    templates = tracker(T, curr_rect, frames, threshold, max_iteration)
      
    return templates

In [ ]:
templates = get_templates(car, templates, 0.001, 200)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82


# Output video

In [ ]:
h, w = car[:,:,0].shape
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('Car_2.mp4', fourcc, 10.0, (w, h))

for i in range(car.shape[2]):
  frame = car[:,:,i].copy()
  output_frame = cv2.rectangle(frame, (templates[i][0][0],templates[i][0][1]), (templates[i][1][0], templates[i][1][1]), (0, 0, 0), 2)
  output_frame = cv2.cvtColor(output_frame, cv2.COLOR_GRAY2RGB)
  out.write(output_frame)

out.release()